### **Building a Rule-based Sentiment Classifier**
This notebook is an attempt to build a rule-based sentiment classifier. It will take in a text X and return a label of "1" if the sentiment of the text is positive, "-1" if the sentiment of the text is negative, and "0" if the sentiment of the text is neutral. You can test the accuracy of your classifier on the Stanford Sentiment Treebank by running the notebook all the way to end.

The final way the classifier decides whether to assign a positive, negative, or neutral label is by calculating the dot product feature_weights * extract_features(X), and if the value is greater than zero, return 1, less than zero return -1, and if exactly zero return 0.


In [7]:
def extract_features(x: str) -> dict[str, float]:
    features = {}
    x_split = x.split(' ')
    
    good_words = ['love', 'good', 'nice', 'great', 'enjoy', 'enjoyed']
    bad_words = ['hate', 'bad', 'terrible', 'disappointing', 'sad', 'lost', 'angry']
    
    for x_words in x_split:
        if x_words in good_words:
            features['good_word_count'] = features.get('good_word_count', 0) + 1
        if x_words in bad_words:
            features['bad_word_count'] = features.get('bad_word_count', 0) + 1 
            
    features['bias'] = 1
    
    return features
    
    feature_weights = {'good_word_count':1.0,'bad_word_count': -1.0, 'bias':0.5 }
            
            

In [8]:
extract_features("I love to play football because it is nice and sad")

{'good_word_count': 2, 'bad_word_count': 1, 'bias': 1}

>From the example above, we can see that the function successfully extracts the number of good and bad words in the input (x)